In [1]:
import pandas as pd
import psycopg2
from rbbm_src.dc_src.DCRepair import (eq_op, 
 non_symetric_op,
#  dc_tuple_violation_template_targeted_t1,
#  dc_tuple_violation_template_targeted_t2
)
import re
from rbbm_src.dc_src.src.classes import parse_rule_to_where_clause
from string import Template

In [2]:
conn=psycopg2.connect('dbname=cr user=postgres')
dc_tuple_violation_template_targeted_t1=\
Template("SELECT DISTINCT $t1_desc,$t2_desc FROM $table t1, $table t2 WHERE $dc_desc")
dc_tuple_violation_template_targeted_t2=\
Template("SELECT DISTINCT $t1_desc,$t2_desc FROM $table t1, $table t2 WHERE $dc_desc")

In [3]:
cols_q = f"select * from flights_new limit 1"
df_row = pd.read_sql(cols_q, conn)

In [11]:
csv_df = pd.read_csv('../../../../flights_dc_finder_input.csv')

In [12]:
csv_df['_tid_']=range(len(csv_df))

,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN,ORIGIN_CITY_NAME,ORIGIN_STATE_ABR,ORIGIN_STATE_NM,DEST_AIRPORT_ID,DEST,DEST_CITY_NAME,DEST_STATE_ABR,DEST_STATE_NM,CRS_DEP_TIME,DEP_TIME,CRS_ARR_TIME,ARR_TIME,ACTUAL_ELAPSED_TIME,DISTANCE
0,4,10/01/2019,MQ,3496,10728,BPT,"Beaumont""/""Port Arthur TX",TX,Texas,11298,DFW,"Dallas""/""Fort Worth TX",TX,Texas,1027,1019,1150,"1138.""0""","79.""0""",270
1,5,25/01/2019,UA,592,13930,ORD,Chicago IL,IL,Illinois,12892,LAX,Los Angeles CA,CA,California,1539,1555,1815,"1806.""0""","251.""0""",1744
2,2,08/01/2019,AA,2615,10849,BZN,Bozeman MT,MT,Montana,11298,DFW,"Dallas""/""Fort Worth TX",TX,Texas,1348,1347,1748,"1733.""0""","166.""0""",1163
3,4,17/01/2019,NK,711,11433,DTW,Detroit MI,MI,Michigan,12889,LAS,Las Vegas NV,NV,Nevada,1923,1920,2102,"2120.""0""","300.""0""",1749
4,2,29/01/2019,OO,3092,12892,LAX,Los Angeles CA,CA,California,14869,SLC,Salt Lake City UT,UT,Utah,1552,1607,1843,"1852.""0""","105.""0""",590
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,5,04/01/2019,WN,575,10693,BNA,Nashville TN,TN,Tennessee,14679,SAN,San Diego CA,CA,California,1350,1417,1620,"1620.""0""","243.""0""",1751
495,2,01/01/2019,EV,4299,11042,CLE,Cleveland OH,OH,Ohio,12264,IAD,Washington DC,VA,Virginia,600,558,724,"703.""0""","65.""0""",288
496,1,28/01/2019,AA,1114,14100,PHL,Philadelphia PA,PA,Pennsylvania,13303,MIA,Miami FL,FL,Florida,1345,1338,1648,"1635.""0""","177.""0""",1013
497,6,12/01/2019,AA,301,12892,LAX,Los Angeles CA,CA,California,11503,EGE,Eagle CO,CO,Colorado,840,836,1204,"1155.""0""","139.""0""",748


In [4]:
cols=list(df_row)

In [5]:
def construct_query_for_violation(role, dc_text, target_table, targeted, cols): 
    predicates = dc_text.split('&')
#     clause = parse_rule_to_where_clause(dc_text)
    constants=[]
    # print(f"t_interest:{t_interest}")
    need_tid=True 
    # if the constraint only has equals, we need to add an artificial
    # key (_tid_) to differentiate tuples in violation with the tuple it
    # self
    for pred in predicates[2:]:
        if(not eq_op.search(pred)):
            need_tid=False
        attr = re.search(r't[1|2]\.([-\w]+)', pred).group(1).lower()
        # print(attr)
#         constants.append(f'{role}.{attr}=\'{t_interest[attr]}\'')
    constants_clause = ' AND '.join(constants)
    # print(f"dc_text:{dc_text}")
    t1_cols=', '.join([f't1.{x} as t1_{x}' for x in cols])
    t2_cols=', '.join([f't2.{x} as t2_{x}' for x in cols])
    
    if(role=='t1'):
        template=dc_tuple_violation_template_targeted_t1
    else:
        template=dc_tuple_violation_template_targeted_t2
    if(targeted):
        r_q  = template.substitute(t1_desc=t1_cols, t2_desc=t2_cols,
                                   table=target_table, dc_desc=parse_rule_to_where_clause(dc_text),
                                           tuple_desc=constants_clause)

    else:
        r_q  = template.substitute(t1_desc=t1_cols, t2_desc=t2_cols,
                                   table=target_table, dc_desc=parse_rule_to_where_clause(dc_text))

        r_q+=f" AND t1._tid_!=t2._tid_"
#     print(r_q)

    return r_q

def find_tuples_in_violation(conn, dc_text, target_table, cols, targeted=False):
    if(non_symetric_op.search(dc_text)):
        q1= construct_query_for_violation('t1', dc_text, target_table, targeted)
        q2 = construct_query_for_violation('t2', dc_text, target_table, targeted)
        res = {'t1': pd.read_sql(q1, conn).to_dict('records'), 
        't2': pd.read_sql(q2, conn).to_dict('records')}
    else:
        q1= construct_query_for_violation('t1', dc_text, target_table, targeted, cols)
        print(q1)
        res = {'t1':pd.read_sql(q1, conn).to_dict('records')}
        # print(res)
    return res 

In [6]:
dc_file='/home/opc/author/RBBM/rbbm_src/muse/data/mas/flights_dcfinder_rules.txt'
table_name='flights_dc_finder_input'
res=[]
rule_texts=[]
try:
    with open(dc_file, "r") as file:
        for line in file:
            rule_texts.append(line.strip())
            # rules_from_line = [(table_name, x) for x in convert_dc_to_muse_rule(line, 'adult', 't1')]
#             rules_from_line = [(table_name, x) for x in convert_dc_to_muse_rule(line, 'flights_new', 't1')]
#             res.extend(rules_from_line)
except FileNotFoundError:
    print("File not found.")
except IOError:
    print("Error reading the file.")


In [7]:
print(len(rule_texts))

83


In [8]:
dirty_tuples = set()

In [9]:
rules_with_stats={r:{} for r in rule_texts}

In [10]:
for r in rule_texts:
    res = find_tuples_in_violation(conn, r, table_name, cols)
    print(f"rule: {r}")
    for k in res:
        for pair in res[k]:
#             if(pair['t1_is_dirty']!=pair['t2_is_dirty']):
            dirty_tuples.add(pair['t1__tid_'])
            dirty_tuples.add(pair['t2__tid_'])
        len_cc = len([x for x in res[k] if (x['t1_is_dirty']==False and x['t2_is_dirty']==False)])
        len_non_cc = len([x for x in res[k] if not (x['t1_is_dirty']==False and x['t2_is_dirty']==False)])
        len_dd = len([x for x in res[k] if (x['t1_is_dirty']==True and x['t2_is_dirty']==True)])
        len_dc = len([x for x in res[k] if ((x['t1_is_dirty']==True and x['t2_is_dirty']==False) or \
                    (x['t1_is_dirty']==False and x['t2_is_dirty']==True))])
        rules_with_stats[r]['len_cc']=len_cc
        rules_with_stats[r]['len_dd']=len_dd
        rules_with_stats[r]['len_dc']=len_dc
        print(f"number of ccs:{len_cc}, number of non ccs: {len_non_cc}, number of dds: {len_dd}, number of dcs:{len_dc}")
#     break
    print('\n')

SELECT DISTINCT t1.day_of_week as t1_day_of_week, t1.fl_date as t1_fl_date, t1.op_unique_carrier as t1_op_unique_carrier, t1.op_carrier_fl_num as t1_op_carrier_fl_num, t1.origin_airport_id as t1_origin_airport_id, t1.origin as t1_origin, t1.origin_city_name as t1_origin_city_name, t1.origin_state_abr as t1_origin_state_abr, t1.origin_state_nm as t1_origin_state_nm, t1.dest_airport_id as t1_dest_airport_id, t1.dest as t1_dest, t1.dest_city_name as t1_dest_city_name, t1.dest_state_abr as t1_dest_state_abr, t1.dest_state_nm as t1_dest_state_nm, t1.crs_dep_time as t1_crs_dep_time, t1.dep_time as t1_dep_time, t1.crs_arr_time as t1_crs_arr_time, t1.arr_time as t1_arr_time, t1.actual_elapsed_time as t1_actual_elapsed_time, t1.distance as t1_distance, t1._tid_ as t1__tid_, t1.is_dirty as t1_is_dirty,t2.day_of_week as t2_day_of_week, t2.fl_date as t2_fl_date, t2.op_unique_carrier as t2_op_unique_carrier, t2.op_carrier_fl_num as t2_op_carrier_fl_num, t2.origin_airport_id as t2_origin_airport_id,

DatabaseError: Execution failed on sql 'SELECT DISTINCT t1.day_of_week as t1_day_of_week, t1.fl_date as t1_fl_date, t1.op_unique_carrier as t1_op_unique_carrier, t1.op_carrier_fl_num as t1_op_carrier_fl_num, t1.origin_airport_id as t1_origin_airport_id, t1.origin as t1_origin, t1.origin_city_name as t1_origin_city_name, t1.origin_state_abr as t1_origin_state_abr, t1.origin_state_nm as t1_origin_state_nm, t1.dest_airport_id as t1_dest_airport_id, t1.dest as t1_dest, t1.dest_city_name as t1_dest_city_name, t1.dest_state_abr as t1_dest_state_abr, t1.dest_state_nm as t1_dest_state_nm, t1.crs_dep_time as t1_crs_dep_time, t1.dep_time as t1_dep_time, t1.crs_arr_time as t1_crs_arr_time, t1.arr_time as t1_arr_time, t1.actual_elapsed_time as t1_actual_elapsed_time, t1.distance as t1_distance, t1._tid_ as t1__tid_, t1.is_dirty as t1_is_dirty,t2.day_of_week as t2_day_of_week, t2.fl_date as t2_fl_date, t2.op_unique_carrier as t2_op_unique_carrier, t2.op_carrier_fl_num as t2_op_carrier_fl_num, t2.or

In [ ]:
dirty_tuples

In [ ]:
len(dirty_tuples)

In [ ]:
rules_with_stats

In [ ]:
sorted(rules_with_stats, key=lambda k: rules_with_stats[k]['len_cc'], reverse=True)

In [ ]:
list({k:v for k,v in rules_with_stats.items() if v['len_cc']==0})